In [1]:
import numpy as np
import multiprocessing as mp
import os
import re
import nibabel as nib
import pickle
import scipy.io as io

In [2]:
# where are the residuals stored?
residual_path = '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm'
roi_path = '/gpfs/scratch/linjjiang/atlases/final_atlas_roi/rsa_atlas_roi'
out_path = '/gpfs/scratch/linjjiang/scan_data/rsa/full_GLM_atlas_roi'
spm_path = '/gpfs/scratch/linjjiang/scan_data/spm/output/spatiotemporal_res'

In [3]:
def list_matching_files(directory,pattern):
    # Define the regular expression pattern
    pattern = re.compile(pattern)
    
    # List all files in the directory
    all_files = os.listdir(directory)
    
    # Filter files that match the pattern
    matching_files = [os.path.join(directory, f) for f in all_files if pattern.match(f)]
    
    # Sort the files by name
    matching_files = sorted(matching_files)
    
    return matching_files

In [4]:
def extract_file_name(path, target_folder):
    # Split the path into parts
    path_parts = path.split(os.sep)
    
    # Find the index of the target folder
    if target_folder in path_parts:
        index = path_parts.index(target_folder)
        
        # Return the next part of the path if it exists
        if index + 1 < len(path_parts):
            return path_parts[index + 1]
    
    return None

In [5]:
# Define a function to categorize the regressors
def categorize_regressors(names):
    task_related = []
    nuisance = []
    for i, name in enumerate(names):
        if ' R' in name or 'constant' in name or 'rp' in name:
            nuisance.append(i)
        else:
            task_related.append(i)
    return task_related, nuisance

In [6]:
files = list_matching_files(residual_path,r'^res_.*\.pkg$')
print(files)

['/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f09_S5752.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f10_S5767.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f10_S5770.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f10_S5774.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f11_S5756.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f11_S5761.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f12_S5777.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f12_S5782.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f15_S5816.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f15_S5820.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f16_S5824.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f16_S5847.pkg', '/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f17_S5895.pkg'

In [7]:
#     # retrieve residual matrix
#     for roi,roi_img in zip(rois,roi_imgs):
            # Use ROI to mask voxels inside the residual map
        # for each subject and session
        # Goal: get a number of residual x number of voxel/channel matrix
        # number of residual: number of time points in total in the GLM
        # number of voxel/channel: the number of voxel inside the ROI


In [8]:
# spawn a pool of parallel workers
p = mp.Pool(processes=28)

In [9]:
for file in files:
    print(file)
    path_parts = file.split(os.sep)
    filename = path_parts[-1]
    #print(filename)

    filenames = re.split(r'[_\.]', filename)
    #print(filenames)

    subject = filenames[1]
    session = filenames[2]
    print(subject,session)
    
    # get the degree of freedom in the model from the SPM.mat
    spm_folder = os.path.join(spm_path,subject,session,'smgs','SPM.mat')
    #print(spm_folder)
    spm = io.loadmat(spm_folder)
    #print(spm[xX][X])
    #dof = xxxx

    SPM = spm['SPM']

    # Get the design matrix
    X = SPM['xX'][0][0]['X'][0][0]

    # Get the names of the regressors
    names_old = SPM['xX'][0][0]['name'][0][0][0]
    names = [name[0] for name in names_old]

    # Categorize the regressors
    task_related_indices, nuisance_indices = categorize_regressors(names)

    # Calculate the number of task-related and nuisance regressors
    num_task_related_regressors = len(task_related_indices)
    num_nuisance_regressors = len(nuisance_indices)
    total_num_regressors = X.shape[1]

    # Display the results
    print(f'Total number of regressors: {total_num_regressors}')
    print(f'Number of task-related regressors: {num_task_related_regressors}')
    print(f'Number of nuisance regressors: {num_nuisance_regressors}')

    # Load residual
    with open(os.path.join('/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm',
                   'res_'+subject+'_'+session+'.pkg'),'rb') as f:
        residuals = pickle.load(f)
        
    #  Calculate degrees of freedom
    num_res = residuals.shape[0]
    dof_task = num_res - num_task_related_regressors
    dof_all = num_res - total_num_regressors
    
    # Get ROI
    roi_filepaths = list_matching_files(roi_path,r'.*\.nii$')
    #print(roi_filepaths)
#     roi_imgs = []
#     rois = []
    for roi_filepath in roi_filepaths:
        roi_img = nib.load(roi_filepath)
        
        path_parts = roi_filepath.split(os.sep)
        filename = path_parts[-1]
        filenames = re.split(r'[.]', filename)
        roi = filenames[0]

        # Get the data from the images
        roi_data = roi_img.get_fdata()
        #print(roi_data.shape)

        # Create a mask from the ROI image (where the value is 1)
        roi_mask = roi_data > 0

        # Find the indices of the voxels that are within the ROI
        roi_indices = np.where(roi_mask)

        # Number of voxels in the ROI
        num_voxels_in_roi = np.sum(roi_mask)

        # Initialize the residual matrix
        residual_mat = np.zeros((residuals.shape[0], num_voxels_in_roi))
        #print(residual_mat)
        
        # Loop through each volume and apply the ROI mask
        for i in range(residuals.shape[0]):
            volume = residuals[i, :, :, :]
            # Extract the values within the ROI
            residual_mat[i, :] = volume[roi_indices]

        if os.path.exists(os.path.join(out_path,subject,'res',session))==False:
            os.makedirs(os.path.join(out_path,subject,'res',session))
               
        # Save
        with open(os.path.join(out_path,subject,'res',session,'residual_'+roi+'.pkg'),'wb') as f:
            pickle.dump([residual_mat,num_res,dof_task,dof_all],f)
        

/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f09_S5752.pkg
f09 S5752
Total number of regressors: 124
Number of task-related regressors: 72
Number of nuisance regressors: 52
/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f10_S5767.pkg
f10 S5767
Total number of regressors: 116
Number of task-related regressors: 72
Number of nuisance regressors: 44
/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f10_S5770.pkg
f10 S5770
Total number of regressors: 95
Number of task-related regressors: 60
Number of nuisance regressors: 35
/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f10_S5774.pkg
f10 S5774
Total number of regressors: 19
Number of task-related regressors: 12
Number of nuisance regressors: 7
/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f11_S5756.pkg
f11 S5756
Total number of regressors: 95
Number of task-related regressors: 60
Number of nuisance regressors: 35
/gpfs/scratch/linjjiang/scan_data/rsa/residual_from_spm/res_f11_

In [10]:
# close the pool of workers
p.close()